# Context
Run the scrape from the latest spine file, but only those which the data is missing

## Libraries

In [1]:
import os
import pandas as pd
from datetime import datetime, timedelta
import re
import time

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException

## Bring in Latest Spine

In [2]:
LATEST_RACELIST_PATH = "/Users/robpacey/Projects/2. Horse Racing/1. Betting Webscrape - Sportinglife/SportingLife/2. Latest RaceList"
STATUS_OUTPUT_PATH = "/Users/robpacey/Projects/2. Horse Racing/1. Betting Webscrape - Sportinglife/SportingLife/3. ScrapeStatus"
DATA_PATHS = {
    "prerace": "/Users/robpacey/Projects/2. Horse Racing/1. Betting Webscrape - Sportinglife/SportingLife/4. Data/1.Prerace",
    "result": "/Users/robpacey/Projects/2. Horse Racing/1. Betting Webscrape - Sportinglife/SportingLife/4. Data/2.Result",
}

In [3]:
def load_latest_status(STATUS_OUTPUT_PATH):
    """Load all status files and return the latest status per race."""
    status_files = [
        os.path.join(STATUS_OUTPUT_PATH, f)
        for f in os.listdir(STATUS_OUTPUT_PATH)
        if f.endswith(".csv")
    ]

    if not status_files:
        print("[INFO] No existing status files found — starting fresh.")
        return pd.DataFrame()

    # Load all status files
    all_status = pd.concat(
        [pd.read_csv(f).assign(SourceFile=os.path.basename(f)) for f in status_files],
        ignore_index=True
    )

    # Extract timestamp (from filename like 20251022_2210_ScrapeStatus.csv)
    all_status["Timestamp"] = all_status["SourceFile"].str.extract(r"(\d{8}_?\d{0,4})")
    all_status["Timestamp"] = pd.to_datetime(all_status["Timestamp"], format="%Y%m%d_%H%M", errors="coerce")

    # Keep the *latest* entry per race based on prerace_URL
    latest_status = (
        all_status.sort_values("Timestamp")
        .groupby("prerace_URL", as_index=False)
        .last()
    )

    print(f"[INFO] Loaded {len(latest_status)} latest race statuses from {len(status_files)} files.")
    return latest_status

In [4]:
def setup_run(num_races=None):
    """
    Prepares a race_df for scraping by:
      - Loading the latest race list
      - Loading all scrape status files and taking latest per race
      - Merging statuses to skip completed races
      - Filtering to 'Pending' only
      - Returning a limited subset (num_races)
    """

    # --- Bring in latest race list ---
    race_files = [os.path.join(LATEST_RACELIST_PATH, f) for f in os.listdir(LATEST_RACELIST_PATH) if f.endswith(".csv")]
    if not race_files:
        raise FileNotFoundError(f"No race list files found in {LATEST_RACELIST_PATH}")

    latest_file = max(race_files, key=os.path.getctime)
    print(f"[INFO] Using latest race list: {latest_file}")
    race_df = pd.read_csv(latest_file)

    # --- Load latest status across all files ---
    status_df = load_latest_status(STATUS_OUTPUT_PATH)

    # --- Merge status into race list ---
    if not status_df.empty:
        race_df = race_df.merge(
            status_df[["prerace_URL", "Status"]],
            on="prerace_URL",
            how="left",
            suffixes=("", "_latest")
        )
        race_df["Status"] = race_df["Status_latest"].fillna(race_df["Status"])
        race_df.drop(columns=["Status_latest"], inplace=True)

    # --- Filter pending only ---
    race_df = race_df[race_df["Status"] == "Pending"].copy()

    # --- Optional: limit number of races ---
    if num_races:
        race_df = race_df.head(num_races)
        print(f"[INFO] Limiting to first {num_races} pending races")

    print(f"[INFO] Ready to scrape {len(race_df)} races")
    return race_df

## Selenium Set Up

In [5]:
def setup_driver():
    """Set up a Chrome WebDriver using the latest cached driver (no re-download)."""
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--disable-gpu")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--window-size=1920,1080")

    # Locate your local WebDriverManager cache folder
    base_path = "/Users/robpacey/.wdm/drivers/chromedriver/mac64"
    versions = sorted(os.listdir(base_path))
    latest_version = versions[-1]  # pick latest
    driver_path = os.path.join(base_path, latest_version, "chromedriver-mac-arm64", "chromedriver")

    service = Service(driver_path)
    driver = webdriver.Chrome(service=service, options=chrome_options)
    return driver

## PreRace Code

In [6]:
def preraceinfo(driver, prerace_url, max_retries=3):
    """Scrape pre-race info and horse-level data from Sporting Life."""
    data = []

    # --- Helper ---
    def safe_text(elem, css=None, attr="text", default="N/A", replace_text=None):
        """Safely extract text/attributes with optional replacements."""
        try:
            if css:
                elem = elem.find_element(By.CSS_SELECTOR, css)
            value = elem.text.strip() if attr == "text" else elem.get_attribute(attr)
            if replace_text:
                for old, new in replace_text.items():
                    value = value.replace(old, new)
            return value
        except:
            return default

    # Retry loop
    for attempt in range(max_retries):
        try:
            driver.get(prerace_url)

            # --- Race-level info ---
            race_name = safe_text(driver, "h1[data-test-id='racecard-race-name']")

            # Date + weekday
            race_date_txt = safe_text(driver, "p[class*='CourseListingHeader__StyledMainSubTitle']")
            try:
                date_obj = datetime.strptime(race_date_txt, "%A %d %B %Y")
                race_date = date_obj.strftime("%d/%m/%Y")
                race_day_of_week = date_obj.strftime("%A")
            except:
                race_date, race_day_of_week = "N/A", "N/A"

            # Time + location
            race_location_txt = safe_text(driver, "p[class*='CourseListingHeader__StyledMainTitle']")
            race_time, race_location = (
                race_location_txt.split(" ", 1) if " " in race_location_txt else ("N/A", race_location_txt)
            )

            # Defaults
            race_class = race_distance = race_going = race_runners = race_surface = winning_time = off_time = "N/A"

            # Loop through additional info parts
            for elem in driver.find_elements(
                By.CSS_SELECTOR, "li.RacingRacecardSummary__StyledAdditionalInfo-sc-ff7de2c2-3"
            ):
                for part in [p.strip() for p in elem.text.split("|")]:

                    if match := re.search(r"Class\s+(\d+)", part):
                        race_class = match.group(1)

                    elif match := re.search(r"Winning time:\s*([0-9m\s\.]+)", part):
                        winning_time = match.group(1).strip()

                    elif match := re.search(r"Off time:\s*([0-9:]+)", part):
                        off_time = match.group(1)

                    elif re.search(r"\d+\s*(m|f|y)", part):
                        race_distance = part

                    elif any(word in part for word in ["Heavy","Soft","Good","Firm","Standard","Yielding","Fast","Slow"]):
                        race_going = part

                    elif match := re.search(r"(\d+)\s*Runners?", part):
                        race_runners = match.group(1)

                    elif re.search(r"(Turf|Allweather|All Weather|AW|Polytrack|Fibresand|Tapeta|Dirt)", part, re.I):
                        race_surface = part

            # --- Horse-level info ---
            horse_containers = driver.find_elements(By.CSS_SELECTOR, "div[class*='Runner__StyledRunnerContainer']")
            for parent in horse_containers:
                try:
                    horse_number = safe_text(parent, "div[data-test-id='saddle-cloth-no']")
                    stall_no = safe_text(parent, "div[data-test-id='stall-no']", default="N/A").strip("()")
                    horse_name = safe_text(parent, "a[data-test-id='horse-name-link']")
                    headgear = safe_text(parent, "sup[data-test-id='headgear']", default="")
                    last_run = safe_text(parent, "sup[data-test-id='last-ran']", default="")

                    commentary = safe_text(parent, "div[data-test-id='commentary']", default="N/A")

                    sub_info_elem = parent.find_element(By.CSS_SELECTOR, "div[data-test-id='horse-sub-info']")
                    sub_info_text = sub_info_elem.text
                    age = re.search(r"Age: (\d+)", sub_info_text)
                    weight = re.search(r"Weight: ([\d-]+)", sub_info_text)

                    jockey_name = safe_text(sub_info_elem, "a[href*='/jockey/'] span", default="N/A").replace("J:", "").strip()
                    trainer_name = safe_text(sub_info_elem, "a[href*='/trainer/'] span", default="N/A").replace("T:", "").strip()

                    odds = safe_text(parent, "span[class*='BetLink']")
                    history_stats = " | ".join(
                        [s.text.strip() for s in parent.find_elements(By.CSS_SELECTOR, "span[data-test-id^='race-history-stat-']")]
                    )

                    data.append({
                        "HorseNumber": horse_number,
                        "StallNumber": stall_no,
                        "HorseName": horse_name,
                        "Headgear": headgear,
                        "LastRun": last_run,
                        "Commentary": commentary,
                        "Age": age.group(1) if age else "",
                        "Weight": weight.group(1) if weight else "",
                        "Jockey": jockey_name,
                        "Trainer": trainer_name,
                        "Odds": odds,
                        "RaceHistoryStats": history_stats,
                        "RaceDate": race_date,
                        "RaceDayOfWeek": race_day_of_week,
                        "RaceLocation": race_location,
                        "RaceName": race_name,
                        "RaceTime": race_time,
                        "RaceClass": race_class,
                        "RaceDistance": race_distance,
                        "RaceGoing": race_going,
                        "RaceRunners": race_runners,
                        "RaceSurface": race_surface,
                        "WinningTime": winning_time,
                        "OffTime": off_time
                    })
                except Exception as e:
                    print("Error parsing horse:", e)

            # --- Non-runners ---
            try:
                non_runners = driver.find_elements(By.CSS_SELECTOR, "div[class^='NonRunner__NonRunnerWrapper']")
                for nr in non_runners:
                    nr_horse_name = safe_text(nr, "[data-test-id='runner-horse-name']", default="N/A").split("<sup")[0].strip()
                    nr_saddle_cloth = safe_text(nr, "[data-test-id='runner-cloth-number']")
                    nr_stall_no = safe_text(nr, "[data-test-id='runner-stall-number']", default="N/A").strip("()")
                    nr_jockey = safe_text(nr, "[data-test-id='runner-jockey-name']")
                    nr_trainer = safe_text(nr, "[data-test-id='runner-trainer-name']")
                    nr_age = safe_text(nr, "[data-test-id='runner-horse-age']", default="", replace_text={"|  Age: ": ""})
                    nr_weight = safe_text(nr, "[data-test-id='runner-horse-weight']", default="", replace_text={"Weight: ": ""})
                    nr_last_run = safe_text(nr, "[data-test-id='runner-horse-last-run']", default="")

                    data.append({
                        "HorseNumber": nr_saddle_cloth,
                        "StallNumber": nr_stall_no,
                        "HorseName": nr_horse_name,
                        "Headgear": "",
                        "LastRun": nr_last_run,
                        "Commentary": "NonRunner",
                        "Age": nr_age,
                        "Weight": nr_weight,
                        "Jockey": nr_jockey,
                        "Trainer": nr_trainer,
                        "Odds": "NonRunner",
                        "RaceHistoryStats": "",
                        "RaceDate": race_date,
                        "RaceDayOfWeek": race_day_of_week,
                        "RaceLocation": race_location,
                        "RaceName": race_name,
                        "RaceTime": race_time,
                        "RaceClass": race_class,
                        "RaceDistance": race_distance,
                        "RaceGoing": race_going,
                        "RaceRunners": race_runners,
                        "RaceSurface": race_surface,
                        "WinningTime": winning_time,
                        "OffTime": off_time
                    })
            except:
                print("No non-runners found")

            break  # ✅ Success, exit retry loop

        except TimeoutException as e:
            print(f"Retry {attempt+1}/{max_retries} failed: {e}")
            if attempt + 1 == max_retries:
                time.sleep(2 * (attempt + 1))
            else:
                raise

    return pd.DataFrame(data)

## PostRace Code

In [7]:
def resultsinfo(driver, result_url, max_retries=3):
    """Scrape post-race results info (race + horses) from Sporting Life."""
    data = []

    # Helper function
    def safe_text(elem, css=None, attr=None, default="N/A"):
        """Safely extract text or attribute from an element or sub-element."""
        try:
            if css:
                elem = elem.find_element(By.CSS_SELECTOR, css)
            return elem.get_attribute(attr) if attr else elem.text.strip()
        except:
            return default

    for attempt in range(max_retries):
        try:
            driver.get(result_url)

            # --- Race-level info ---
            try:
                race_name = safe_text(driver, "h1[data-test-id='racecard-race-name']")

                race_date_text = safe_text(driver, "p[class*='CourseListingHeader__StyledMainSubTitle']")
                try:
                    race_date_obj = datetime.strptime(race_date_text, "%A %d %B %Y")
                    race_date = race_date_obj.strftime("%d/%m/%Y")
                    race_day_of_week = race_date_obj.strftime("%A")
                except:
                    race_date = race_day_of_week = "N/A"

                race_time_text = safe_text(driver, "p[class*='CourseListingHeader__StyledMainTitle']")
                parts = race_time_text.split()
                race_time = parts[0] if parts else "N/A"
                race_location = " ".join(parts[1:]) if len(parts) > 1 else "N/A"

                # Defaults
                winning_time = race_distance = race_going = race_runners = race_surface = "N/A"

                # Additional info
                for li in driver.find_elements(
                    By.CSS_SELECTOR,
                    "li.RacingRacecardSummary__StyledAdditionalInfo-sc-ff7de2c2-3"
                ):
                    text = li.text.strip()

                    # Winning time
                    match = re.search(r"Winning time:\s*([0-9m\s\.]+)", text)
                    if match:
                        winning_time = match.group(1).strip()
                        continue

                    # Distance
                    match = re.search(r"(\d+\s*(?:m|f|y)(?:\s*\d*\s*(?:f|y))?)", text)
                    if match:
                        race_distance = match.group(1)

                    # Going
                    match = re.search(
                        r"(Heavy|Soft|Good to Soft|Good to Firm|Good|Firm|Standard|Standard / Slow|Yielding|Fast|Slow)",
                        text,
                    )
                    if match:
                        race_going = match.group(1)

                    # Runners
                    match = re.search(r"(\d+)\s*Runners?", text)
                    if match:
                        race_runners = match.group(1)

                    # Surface
                    match = re.search(r"(Turf|All Weather|AW|Allweather|Polytrack|Fibresand|Tapeta|Dirt)", text, re.I)
                    if match:
                        race_surface = match.group(1)

            except Exception as e:
                print("Error extracting race info:", e)
                race_name = race_date = race_day_of_week = race_location = race_time = \
                race_distance = race_going = race_runners = race_surface = winning_time = "N/A"

            # --- Horse-level info ---
            horse_elements = driver.find_elements(
                By.CSS_SELECTOR, "div[class*='ResultRunner__StyledResultRunnerWrapper']"
            )

            for horse_elem in horse_elements:
                try:
                    pos = safe_text(horse_elem, "div[data-test-id='position-no']")
                    silk_url = safe_text(horse_elem, "div[class*='StyledSilkContainer'] img", attr="src")
                    horse_number = safe_text(horse_elem, "div[data-test-id='saddle-cloth-no']")
                    stall_number = safe_text(horse_elem, "div[data-test-id='stall-no']")
                    horse_name = safe_text(horse_elem, "div[class*='StyledHorseName'] a")

                    ride_description = safe_text(horse_elem, "div[data-test-id='ride-description']", default="N/A")

                    # Trainer & Jockey
                    trainer = jockey = "N/A"
                    for span in horse_elem.find_elements(By.CSS_SELECTOR, "span[class*='StyledPersonName']"):
                        try:
                            label = span.find_element(By.XPATH, "./..").text
                            if label.startswith("T:"):
                                trainer = span.text.strip()
                            elif label.startswith("J:"):
                                jockey = span.text.strip()
                        except:
                            continue

                    # Starting Price (SP)
                    sp = safe_text(horse_elem, "span[class*='BetLink__BetLinkStyle']")

                    # Prize Money (using XPath relative to position number)
                    try:
                        prize_elem = driver.find_element(
                            By.XPATH,
                            f"//span[contains(text(), '{pos}')]//following-sibling::span[contains(@class,'PrizeNumber')]"
                        )
                        prize_money = prize_elem.text.strip()
                    except:
                        prize_money = "0"

                    data.append({
                        "Pos": pos,
                        "SilkURL": silk_url,
                        "HorseNumber": horse_number,
                        "StallNumber": stall_number,
                        "HorseName": horse_name,
                        "Result": pos,
                        "SP": sp,
                        "Trainer": trainer,
                        "Jockey": jockey,
                        "PrizeMoney": prize_money,
                        "RideDescription": ride_description,
                        "RaceDate": race_date,
                        "RaceDayOfWeek": race_day_of_week,
                        "RaceLocation": race_location,
                        "RaceName": race_name,
                        "RaceTime": race_time,
                        "WinningTime": winning_time,
                        "RaceDistance": race_distance,
                        "RaceGoing": race_going,
                        "RaceRunners": race_runners,
                        "RaceSurface": race_surface,
                    })

                except Exception as e:
                    print("Error parsing horse:", e)

            break  # success → break retry loop

        except TimeoutException as e:
            print(f"Retry {attempt+1}/{max_retries} failed: {e}")
            if attempt + 1 == max_retries:
                time.sleep(2 * (attempt + 1))
            else:
                raise

    return pd.DataFrame(data)

## Loop through all races

In [ ]:
def scrape_all_races(race_df):
    """Loop through each race and run pre- and post-race scraping with live progress and ETA."""
    driver = setup_driver()
    today_str = datetime.today().strftime("%Y%m%d_%H%M")

    prerace_data = []
    postrace_data = []
    updated_rows = []

    race_df = race_df.reset_index(drop=True)
    total_races = len(race_df)
    print(f"[INFO] Starting scrape for {total_races} races...\n")
    start_time = time.time()

    for idx, row in race_df.iterrows():
        race_num = idx + 1
        prerace_url = row["prerace_URL"]
        postrace_url = row["postrace_URL"]
        location = row.get("Location", "N/A")
        time_ = row.get("Time", "N/A")
        status = row.get("Status", "Pending")

        # Skip already completed or abandoned races
        if status in ["Complete", "Abandoned"]:
            updated_rows.append(row)
            print(f"[{race_num}/{total_races}] ⏩ Skipped ({status}): {location} {time_}")
            continue

        race_success = False
        race_start = time.time()

        # --- Pre-race scrape ---
        if pd.notna(prerace_url) and prerace_url.strip():
            try:
                df_pre = preraceinfo(driver, prerace_url)
                if not df_pre.empty:
                    df_pre["SourceURL"] = prerace_url
                    prerace_data.append(df_pre)
                    race_success = True
            except Exception as e:
                print(f"[{race_num}/{total_races}] ❌ Error scraping prerace: {e}")

        # --- Post-race scrape ---
        if pd.notna(postrace_url) and postrace_url.strip():
            try:
                df_post = resultsinfo(driver, postrace_url)
                if not df_post.empty:
                    df_post["SourceURL"] = postrace_url
                    postrace_data.append(df_post)
                    race_success = True
            except Exception as e:
                print(f"[{race_num}/{total_races}] ❌ Error scraping postrace: {e}")

        # --- Update Status ---
        if race_success:
            row["Status"] = "Complete"
            outcome = "✅ Completed"
        else:
            row["Status"] = "Abandoned" if "abandoned" in prerace_url.lower() else "Pending"
            outcome = "⚠️ No data found"

        updated_rows.append(row)

        # --- ETA + progress display ---
        race_end = time.time()
        elapsed = race_end - start_time
        avg_per_race = elapsed / race_num
        remaining = (total_races - race_num) * avg_per_race
        eta_time = datetime.now() + timedelta(seconds=remaining)

        print(f"[{race_num}/{total_races}] {outcome}: {location} {time_} | "
              f"⏱️ {avg_per_race:.1f}s avg | ETA {eta_time.strftime('%H:%M:%S')}")

    driver.quit()

    total_time = time.time() - start_time
    complete_count = sum(1 for row in updated_rows if row["Status"] == "Complete")
    print(f"\n[INFO] Scraping complete → {complete_count}/{total_races} completed "
          f"in {total_time/60:.1f} minutes ✅")

    # --- Combine each into single DataFrame ---
    df_PreRace = pd.concat(prerace_data, ignore_index=True) if prerace_data else pd.DataFrame()
    df_PostRace = pd.concat(postrace_data, ignore_index=True) if postrace_data else pd.DataFrame()

    # --- Save updated status tracking ---
    latest_status_df = pd.DataFrame(updated_rows)

    STATUS_OUTPUT_PATH = "/Users/robpacey/Projects/2. Horse Racing/1. Betting Webscrape - Sportinglife/SportingLife/3. ScrapeStatus"
    os.makedirs(STATUS_OUTPUT_PATH, exist_ok=True)
    status_filename = os.path.join(STATUS_OUTPUT_PATH, f"{
        str}_ScrapeStatus.csv")

    latest_status_df.to_csv(status_filename, index=False)
    print(f"[INFO] Status file saved → {status_filename}")

    # --- Save scraped data ---
    prerace_filename = os.path.join(DATA_PATHS["prerace"], f"{today_str}_prerace.csv")
    results_filename = os.path.join(DATA_PATHS["result"], f"{today_str}_results.csv")

    df_PreRace.to_csv(prerace_filename, index=False)
    df_PostRace.to_csv(results_filename, index=False)

    print(f"[INFO] ✅ Prerace data saved → {prerace_filename}")
    print(f"[INFO] ✅ Results data saved → {results_filename}")

    return df_PreRace, df_PostRace

## Execute

In [37]:
# --- Create race list of x many races ---
race_df = setup_run(num_races=1000)

[INFO] Using latest race list: /Users/robpacey/Projects/2. Horse Racing/1. Betting Webscrape - Sportinglife/SportingLife/2. Latest RaceList/20251027_RaceLists.csv
[INFO] Loaded 23824 latest race statuses from 36 files.
[INFO] Limiting to first 1000 pending races
[INFO] Ready to scrape 0 races


In [36]:
# --- Execute Code ---
df_PreRace, df_PostRace = scrape_all_races(race_df)

[INFO] Starting scrape for 1000 races...

[1/1000] ✅ Completed: Ascot 16:40 | ⏱️ 5.3s avg | ETA 14:41:55
[2/1000] ✅ Completed: Dundalk 16:45 | ⏱️ 4.1s avg | ETA 14:22:26
[3/1000] ✅ Completed: Catterick 10:30 | ⏱️ 3.4s avg | ETA 14:09:59
[4/1000] ✅ Completed: Catterick 11:28 | ⏱️ 3.1s avg | ETA 14:04:42
[5/1000] ✅ Completed: Catterick 11:56 | ⏱️ 3.0s avg | ETA 14:03:31
[6/1000] ✅ Completed: Catterick 12:23 | ⏱️ 2.9s avg | ETA 14:01:39
[7/1000] ✅ Completed: Catterick 12:50 | ⏱️ 2.8s avg | ETA 14:00:06
[8/1000] ✅ Completed: Catterick 13:20 | ⏱️ 2.8s avg | ETA 13:59:42
[9/1000] ✅ Completed: Catterick 13:55 | ⏱️ 2.8s avg | ETA 13:59:48
[10/1000] ✅ Completed: Newton Abbot 14:15 | ⏱️ 2.8s avg | ETA 13:59:16
[11/1000] ✅ Completed: Newton Abbot 14:54 | ⏱️ 2.7s avg | ETA 13:58:34
[12/1000] ✅ Completed: Newton Abbot 15:35 | ⏱️ 2.7s avg | ETA 13:57:53
[13/1000] ✅ Completed: Punchestown 15:05 | ⏱️ 2.7s avg | ETA 13:58:03
[14/1000] ✅ Completed: Newton Abbot 16:15 | ⏱️ 2.7s avg | ETA 13:57:38
[15/100